# Lecture 3 – CSCI 3022


[Acknowledgments Page](https://ds100.org/fa23/acks/)

A demonstration of advanced `pandas` syntax to accompany Lecture 3.

In [1]:
# Linear algebra, probability
import numpy as np

# Data manipulation
import pandas as pd

# Interactive visualization library
import plotly.express as px

In [ ]:
# Loading the elections DataFrame
elections = pd.read_csv("data/elections.csv")

elections.head() 

## Data Extraction in `Pandas`

### Context-dependent Extraction using `[]`

`[]` is *context-dependent*.

`[]` only takes one argument, which may be:
1. A slice of row integers (i.e. something of the form start:stop, where the slice is exclusive of the stop value)
2. A list of column labels.
3. A single column label.


If we provide a slice of row numbers, we get the numbered rows.

In [ ]:
elections[3:7]

In [ ]:
elections[:3]

In [ ]:
elections[7:]

If we provide a list of column names, we get the listed columns.

In [ ]:
elections[["Year", "Candidate", "Result"]]

And if we provide a single column name we get back just that column, stored as a `Series`.

In [ ]:
elections["Candidate"]

In [ ]:
#Notice above this returned a Series.  If we wanted to return a 1-column DataFrame we could use:
elections[["Candidate"]]

In [ ]:
elections[:3][["Year","Candidate"]]

## Other Methods for Slicing:

#### Label-Based Extraction Using`loc`

Arguments to `.loc` can be:
1. A list.
2. A slice (syntax is inclusive of the right-hand side of the slice).
3. A single value.


`loc` selects items by row and column *label*.

In [ ]:
# Selection by a list
elections.loc[[87, 25, 179], ["Year", "Candidate", "Result"]]

In [ ]:
# Selection by a list and a slice of columns
elections.loc[[87, 25, 179], "Popular vote":"%"]

In [ ]:
# Extracting all rows using a colon
elections.loc[:, ["Year", "Candidate", "Result"]]

In [ ]:
# Extracting all columns using a colon
elections.loc[[87, 25, 179], :]

In [ ]:
# Selection by a list and a single-column label
elections.loc[[87, 25, 179], "Popular vote"]

In [ ]:
# Note that if we pass "Popular vote" in a list, the output will be a DataFrame
elections.loc[[87, 25, 179], ["Popular vote"]]

In [ ]:
# Selection by a row label and a column label
elections.loc[0, "Candidate"]

### Integer-Based Extraction Using `iloc`

`iloc` selects items by row and column *integer* position.

Arguments to `.iloc` can be:
1. A list.
2. A slice (syntax is exclusive of the right hand side of the slice).
3. A single value.


In [ ]:
# Select the rows at positions 1, 2, and 3.
# Select the columns at positions 0, 1, and 2.
# Remember that Python indexing begins at position 0!
elections.iloc[[1, 2, 3], [0, 1, 2]]

In [ ]:
# Index-based extraction using a list of rows and a slice of column indices
elections.iloc[[1, 2, 3], 0:3]

In [ ]:
# Selecting all rows using a colon
elections.iloc[:, 0:3]

In [ ]:
elections.iloc[[1, 2, 3], 1]

In [ ]:
# Extracting the value at row 0 and the second column
elections.iloc[0,1]

## Dataset: California baby names


In today's lecture, we will download data from the United States Social Security office containing the number of registered names broken down by **year**, **sex**, and **name**. This is often called the Baby Names Data as social security numbers (SSNs) are typically given at birth.

The cell below pulls census data from a government website and then loads it into a usable form and then extracts the information for babies born in California into the `babynames` dataframe.  


The code shown here is outside of the scope of CSCI 3022, but you're encouraged to dig into it if you are interested!

### Understanding the Setting

**In CSCI 3022 you will have to learn about different data sources (and their limitations) on your own.**

Reading from [SSN Office description](https://www.ssa.gov/oact/babynames/background.html), bolded for readability: 


> All names are from Social Security card applications for **births that occurred in the United States** after 1879. **Note that many people born before 1937 never applied** for a Social Security card, so their names are not included in our data. For others who did apply, our records may not show the place of birth, and again their names are not included in our data.

> **To safeguard privacy, we exclude** from our tabulated lists of names those that would indicate, or would allow the ability to determine, **names with fewer than 5 occurrences** in any geographic area. If a name has less than 5 occurrences for a year of birth in any state, the sum of the state counts for that year will be less than the national count.

> All data are from a **100% sample** of our records on Social Security card applications as of March 2023.

In [ ]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "data/babynamesbystate.zip"
if not os.path.exists(local_filename): # If the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'STATE.CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.head()

## Conditional Selection

In [ ]:
# Ask yourself: Why is :9 is the correct slice to select the first 10 rows?
babynames_first_10_rows = babynames.loc[:9, :]

babynames_first_10_rows

By passing in a sequence (list, array, or `Series`) of boolean values, we can extract a subset of the rows in a `DataFrame`. We will keep *only* the rows that correspond to a boolean value of `True`.

In [ ]:
# Notice how we have exactly 10 elements in our boolean array argument.
babynames_first_10_rows[[True, False, True, False, True, False, True, False, True, False]]

In [ ]:
# Or using .loc to filter a DataFrame by a Boolean array argument
babynames_first_10_rows.loc[[True, False, True, False, True, False, True, False, True, False], :]


Oftentimes, we'll use boolean selection to check for entries in a `DataFrame` that meet a particular condition.

In [ ]:
# First, use a logical condition to generate a boolean Series
logical_operator = (babynames["Sex"] == "F")
logical_operator

In [ ]:
# Then, use this boolean Series to filter the DataFrame
babynames[logical_operator]

Boolean selection also works with `loc`!

In [ ]:
# Notice that we did not have to specify columns to select 
# If no columns are referenced, pandas will automatically select all columns
babynames.loc[babynames["Sex"] == "F"]

### Bitwise Operators

To filter on multiple conditions, we combine boolean operators using **bitwise comparisons**.

Symbol | Usage      | Meaning 
------ | ---------- | -------------------------------------
~    | ~p       | Returns negation of p
&#124; | p &#124; q | p OR q
&    | p & q    | p AND q
^  | p ^ q | p XOR q (exclusive or)

### `.isin` for Selection based on a list, array, or `Series` 

In [ ]:
# Note: The parentheses surrounding the code make it possible to break the code into multiple lines for readability

(
    babynames[(babynames["Name"] == "Bella") | 
              (babynames["Name"] == "Alex") |
              (babynames["Name"] == "Narges") |
              (babynames["Name"] == "Lisa")]
)


In [ ]:
# A more concise method to achieve the above: .isin
names = ["Bella", "Alex", "Narges", "Lisa"]
display(babynames["Name"].isin(names))
display(babynames[babynames["Name"].isin(names)])

### `.str` Functions for Defining a Condition

In [ ]:
# What if we only want names that start with "N"?
display(babynames["Name"].str.startswith("N"))
display(babynames[babynames["Name"].str.startswith("N")])

## Adding, Removing, and Modifying Columns

### Add a Column
To add a column, use `[]` to reference the desired new column, then assign it to a `Series` or array of appropriate length.

In [ ]:
# Create a Series of the length of each name
babyname_lengths = babynames["Name"].str.len()

# Add a column named "name_lengths" that includes the length of each name
babynames["name_lengths"] = babyname_lengths

babynames

### Modify a Column
To modify a column, use `[]` to access the desired column, then re-assign it to a new array or Series.

In [ ]:
# Modify the "name_lengths" column to be one less than its original value
babynames["name_lengths"] = babynames["name_lengths"] - 1
babynames

### Rename a Column Name
Rename a column using the `.rename()` method.

In [ ]:
# Rename "name_lengths" to "Length"
babynames = babynames.rename(columns={"name_lengths":"Length"})
babynames

### Delete a Column
Remove a column using `.drop()`.

In [ ]:
# Remove our new "Length" column
babynames = babynames.drop("Length", axis="columns")
babynames

## Useful Utility Functions

### `NumPy`

 `NumPy`   [functions](https://www.data8.org/su23/reference/#array-functions-and-methods) are compatible with objects in `pandas`. 

In [ ]:
yash_counts = babynames[babynames["Name"] == "Yash"]["Count"]
yash_counts

In [ ]:
# Average number of babies named Yash each year

np.mean(yash_counts)

In [ ]:
# Max number of babies named Yash born in any single year

max(yash_counts)

### Built-In `pandas` Methods

There are many, *many* utility functions built into `pandas`, far more than we can possibly cover in lecture. You are encouraged to explore all the functionality outlined in the `pandas` [documentation](https://pandas.pydata.org/docs/reference/index.html).

In [ ]:
# Returns the shape of the object in the format (num_rows, num_columns)
babynames.shape

In [ ]:
# Returns the total number of entries in the object, equal to num_rows * num_columns
babynames.size

In [ ]:
# What summary statistics can we describe?
babynames.describe()

In [ ]:
# Our statistics are slightly different when working with a Series
babynames["Sex"].describe()

In [ ]:
# Randomly sample row(s) from the DataFrame
babynames.sample()

In [ ]:
# Rerun this cell a few times – you'll get different results!
babynames.sample(5).iloc[:, 2:]

In [ ]:
# Sampling with replacement
babynames[babynames["Year"] == 2000].sample(4, replace = True).iloc[:,2:]

In [ ]:
# Count the number of times each unique value occurs in a Series
babynames["Name"].value_counts()

In [ ]:
# Return an array of all unique values in the Series
babynames["Name"].unique()

In [ ]:
# Sort a Series
babynames["Name"].sort_values()

In [ ]:
# Sort a DataFrame – there are lots of Michaels in California
babynames.sort_values(by = "Count", ascending = False)

In [ ]:
#Sorting by multiple columns

babynames.sort_values(by = ["Count","Name"])

#babynames.sort_values(by = ["Name","Count"])

In [ ]:
babynames.query('Sex == "M" and Year ==2020').sort_values("Name", ascending=False)

## Custom sorting

### Approach 1: Create a temporary column

In [ ]:
# Create a Series of the length of each name
babyname_lengths = babynames["Name"].str.len()

# Add a column named "name_lengths" that includes the length of each name
babynames["name_lengths"] = babyname_lengths
babynames.head(5)

In [ ]:
# Sort by the temporary column
babynames = babynames.sort_values(by = "name_lengths", ascending=False)
babynames.head(5)

In [ ]:
# Drop the 'name_length' column
babynames = babynames.drop("name_lengths", axis = 'columns')
babynames.head(5)

### Approach 2: Sorting using the `key` argument

In [ ]:
def yourfunctionname(x):
    return x.str.len()

(
babynames.sort_values("Name",key=yourfunctionname,ascending=False)
        .head()
)

In [ ]:
babynames.sort_values("Name", key = lambda x:x.str.len(), ascending = False).head()

### Approach 3: Sorting Using the `map` Function

We can also use the Python map function if we want to use an arbitrarily defined function. Suppose we want to sort by the number of occurrences of "dr" plus the number of occurences of "ea".

In [ ]:
# First, define a function to count the number of times "dr" or "ea" appear in each name
def dr_ea_count(string):
    return string.count('dr') + string.count('ea')

# Then, use `map` to apply `dr_ea_count` to each name in the "Name" column
babynames["dr_ea_count"] = babynames["Name"].map(dr_ea_count)

# Sort the DataFrame by the new "dr_ea_count" column so we can see our handiwork
babynames = babynames.sort_values(by = "dr_ea_count", ascending=False)
babynames.head()

In [ ]:
# Drop the `dr_ea_count` column
babynames = babynames.drop("dr_ea_count", axis = 'columns')
babynames.head(5)

## Grouping

Group rows that share a common feature, then aggregate data across the group.

In this example, we count the total number of babies born in each year (considering only a small subset of the data, for simplicity).

<img src="images/groupby.png" width="800"/>

There are many different aggregation functions we can use, all of which are useful in different applications.

In [ ]:
# What is the earliest year in which each name appeared?

babynames.groupby("Name")[["Year"]].agg(min)

In [ ]:
babynames.groupby("Name").agg(min)

In [ ]:
# What is the largest single-year count of each name?
babynames.groupby("Name")[["Count"]].agg(max)

In [ ]:
babynames.groupby("Name")[["Count"]].last()

## Case Study: Name "Popularity"

In this exercise, let's find the name with sex "F" that has dropped most in popularity since its peak usage. We'll start by filtering `babynames` to only include names corresponding to sex "F".

In [ ]:
f_babynames = babynames[babynames["Sex"] == "F"]
f_babynames

In [ ]:
# We sort the data by year
f_babynames = f_babynames.sort_values("Year")
f_babynames

To build our intuition on how to answer our research question, let's visualize the prevalence of the name "Jennifer" over time.

In [ ]:
# We'll talk about how to generate plots in a later lecture
fig = px.line(f_babynames[f_babynames["Name"] == "Jennifer"],
              x = "Year", y = "Count")
fig.update_layout(font_size = 18, 
                  autosize=False, 
                  width=1000, 
                  height=400)

We'll need a mathematical definition for the change in popularity of a name.

Define the metric "ratio to peak" (RTP). We'll calculate this as the count of the name in 2022 (the most recent year for which we have data) divided by the largest count of this name in *any* year. 

A demo calculation for Jennifer:

In [ ]:
# In the year with the highest Jennifer count, 6065 Jennifers were born
max_jenn = np.max(f_babynames[f_babynames["Name"] == "Jennifer"]["Count"])
max_jenn

In [ ]:
# Remember that we sorted f_babynames by year. 
# This means that grabbing the final entry gives us the most recent count of Jennifers: 114
# In 2022, the most recent year for which we have data, 114 Jennifers were born
curr_jenn = f_babynames[f_babynames["Name"] == "Jennifer"]["Count"].iloc[-1]
curr_jenn

In [ ]:
# Compute the RTP
curr_jenn / max_jenn

We can also write a function that produces the `ratio_to_peak`for a given `Series`. This will allow us to use `.groupby` to speed up our computation for all names in the dataset.

In [ ]:
def ratio_to_peak(series):
    """
    Compute the RTP for a Series containing the counts per year for a single name
    """
    return series.iloc[-1] / np.max(series)

In [ ]:
# Construct a Series containing our Jennifer count data
jenn_counts_ser = f_babynames[f_babynames["Name"] == "Jennifer"]["Count"]

# Then, find the RTP
ratio_to_peak(jenn_counts_ser)

Now, let's use `.groupby` to compute the RTPs for *all* names in the dataset.

You may see a warning message when running the cell below. As discussed in lecture, `pandas` can't apply an aggregation function to non-numeric data (it doens't make sense to divide "CA" by a number). By default, `.groupby` will drop any columns that cannot be aggregated.

In [ ]:
rtp_table = f_babynames.groupby("Name")[["Year", "Count"]].agg(ratio_to_peak)
rtp_table

In [ ]:
# Results in a TypeError
#rtp_table = f_babynames.groupby("Name").agg(ratio_to_peak)
# rtp_table

To avoid the warning message above, we explicitly extract only the columns relevant to our analysis before using `.agg`.

In [ ]:
# Recompute the RTPs, but only performing the calculation on the "Count" column
rtp_table = f_babynames.groupby("Name")[["Count"]].agg(ratio_to_peak)
rtp_table

In [ ]:
# Rename "Count" to "Count RTP" for clarity
rtp_table = rtp_table.rename(columns = {"Count": "Count RTP"})
rtp_table

In [ ]:
# What name has fallen the most in popularity?
rtp_table.sort_values("Count RTP")

We can visualize the decrease in the popularity of the name "Debra:"

In [ ]:
def plot_name(*names):
    fig = px.line(f_babynames[f_babynames["Name"].isin(names)], 
                  x = "Year", y = "Count", color="Name",
                  title=f"Popularity for: {names}")
    fig.update_layout(font_size = 18, 
                  autosize=False, 
                  width=1000, 
                  height=400)
    return fig

plot_name("Debra")

In [ ]:
# Find the 10 names that have decreased the most in popularity
top10 = rtp_table.sort_values("Count RTP").head(10).index
top10

In [ ]:
plot_name(*top10)

For fun, try plotting your name or your friends' names.

### iClicker Exercises
We want to obtain the rows with the first three baby names with `count > 250`. 

In [ ]:
babynames.loc[babynames["Count"] > 250, :].head(3)

In [ ]:
babynames.loc[babynames["Count"] > 250, :].iloc[0:2, :]

In [ ]:
babynames[babynames["Count"] > 250].head(3)

In [ ]:
babynames.loc[babynames["Count"] > 250, :].iloc[0:3, :]

In [ ]:
# The code below uses the full babynames dataset, which is why some numbers are different relative to the diagram

babynames.groupby("Year").agg(sum)

#babynames.groupby("Year")[["Count"]].agg(sum)